# Data Science Project

* Name: Author Name
* Email:


## TABLE OF CONTENTS 


- **[Introduction](#INTRODUCTION)<br>**
- **[OBTAIN](#OBTAIN)**<br>
- **[SCRUB](#SCRUB)**<br>
- **[EXPLORE](#EXPLORE)**<br>
- **[MODEL](#MODEL)**<br>
- **[iNTERPRET](#iNTERPRET)**<br>
- **[Conclusions/Recommendations](#CONCLUSIONS-&-RECOMMENDATIONS)<br>**
___

# INTRODUCTION

> Explain the point of your project and what question you are trying to answer with your modeling.



# OBTAIN

**If you are running this notebook without restarting the kernel replace '%load_ext autoreload' in imports with '%reload_ext autoreload'**

## Imports and Functions

In [ ]:
# Importing packages
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import statsmodels
import statsmodels.tsa.api as tsa
import plotly.express as px
import plotly.io as pio
import math
from math import sqrt
import holidays
import pmdarima as pm

from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error

from pmdarima.arima.stationarity  import ADFTest
from pmdarima.arima.utils import ndiffs
from pmdarima.arima.utils import nsdiffs

import pickle
import os
import json

from pathlib import Path
import subprocess
import io
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning)

from functions_all import *

%load_ext autoreload
%autoreload 2
%matplotlib inline

## Data

### Data source and data description

Data is from FBI Crime Data Explorer
[NIBRS data for Colorado from 2009-2019](https://crime-data-explorer.fr.cloud.gov/pages/downloads)

The [data dictionary](data/NIBRS_DataDictionary.pdf) is  and a [record descriptiopn](data/NIBRS_Record_Description.pdf) are available.


The description of the main and reference tables is in data/README.md file.
The agency implemented some changes to the files structure in 2016 and removed the sqlite create and load scripts from the zip directories.
Another fact worth mentioning is that files 'nibrs_property_desc.csv' from 2014 and 2015 have duplicated nibrs_property_desc_ids (unique identifier in the nibrs_property_desc table) which complicated the loading of the data.

The rest of the original data description is in description is in the [notebook](capstone_prj_scrub_part1.ipynb) with the first part of data pre-processing.

### Using an already created sqlite database

The notebook with database creation is [here](creating_sqlite_db.ipynb). The referenced database is in ***data/sqlite/db/production1 db***. It takes 2.5 minutes to run the database creation script.

# SCRUB

## Part I, pre-processing the data in SQL database

The first part of the scrubbing process (working with sqlite3 database, production1) is in [this notebook](capstone_prj_scrub_part1.ipynb). It takes about 12 minutes to run the code in part1 notebook. The following code is using dataframes created in part I.

In part I the following dataframes have been created and saved in the pickle files:<br>

    1. df_incident: data/pickled_dataframes/incident.pickle; main incident DF with date/time of an incident
    2. df_offense: data/pickled_dataframes/offense.pickle: main offense DF with offense names and categories
    3. df_offender: data/pickled_dataframes/offender.pickle; main offender DF with demographic info
    4. df_victim: data/pickled_dataframes/victim.pickle; main victim DF with demographic info
    5. df_weapon: data/pickled_dataframes/weapon.pickle; main weapon DF with a weapon category used in an offense
    6. df_bias: data/pickled_dataframes/bias.pickle; main bias DF with offense bias motivation
    7. df_rel: data/pickled_dataframes/relationship.pickle; main victim-offender relationship DF with relationship category
    

## Part II, scrubbing the data in DataFrames

<br><br><span style="font-size:1.2em;">The next step is uploading, displaying info and scrubbing the dataframes in [part 2 notebook](capstone_prj_scrub_part2-px.ipynb) </span>

<br><br><span style="font-size:1.2em;"><b>1. Offense, incident, bias and weapon DataFrames are combined into one for the Times-series analysis<br>
2. Offender, victim and relationship DataFrames are set aside for the future dashboard.</b></span><br><br>
<span style="font-size:1.2em;">The cleaned-up dataframes can be found here:</span>

    1. df_incident: data/pickled_dataframes/incident_clean.pickle; main cleaned-up incident DF with date/time of an incident
    2. df_offense: data/pickled_dataframes/offense_clean.pickle: main cleaned-up offense DF with offense names and categories
    3. df_offender: data/pickled_dataframes/offender_clean.pickle; main cleaned-up offender DF with demographic info
    4. df_victim: data/pickled_dataframes/victim_clean.pickle; main cleaned-up victim DF with demographic info
    5. df_weapon: data/pickled_dataframes/weapon_clean.pickle; main cleaned-up weapon DF with a weapon category used in an offense
    6. df_bias: data/pickled_dataframes/bias_clean.pickle; main cleaned-up bias DF with offense bias motivation
    7. df_rel: data/pickled_dataframes/rel_clean.pickle; main cleaned-up victim-offender relationship DF with relationship category
    

### Combining Incident, Offense, Bias and Weapon DataFrames

<br><br><span style="font-size:1.2em;">The cleaned-up FULL dataframes can be found here:</span>

    df_full: 'data/pickled_dataframes/df_full_clean.pickle'; main cleaned-up combined DF for time-series analysis

In [ ]:
with open('data/pickled_dataframes/df_full_clean.pickle', 'rb') as f:
    df_full=pickle.load(f)

# EXPLORE

## EDA

<span style="font-size:1.2em; background: lightblue">All EDA part is in the notebook with Part II [part 2 notebook](capstone_prj_scrub_part2-px.ipynb)</span>

<br><br><span style="font-size:1.2em;">The dictionaries with timeseries of varios crime categories and crime locations can be fount here:</span>

    1. TS_crime_categoryt: data/pickled_ts_dict/TS_crime_category.pickle; a dictionary with crime categories and associated time series
    2. TS_crime_against: data/pickled_ts_dict/TS_crime_against.pickle: a dictionary with crime against categories and associated time series
    3. TS_crime_location: data/pickled_ts_dict/TS_crime_location.pickle; a dictionary with crime locations and associated time series
   

# MODEL

## Splitting into a training and a test sets

<br><span style="font-size:1.2em;">I am cutting off a ~10% tail of my data to create a test set.</span><br>

In [ ]:
with open('data/pickled_ts/ts_weekly.pickle', 'rb') as f:
    ts_weekly=pickle.load(f)

In [ ]:
train_size = round(len(ts_weekly) * 0.90)
ts_train, ts_test = ts_weekly[:train_size], ts_weekly[train_size:]
print('Observations: %d weeks' % (len(ts_weekly)))
print('Training Observations: %d weeks' % (len(ts_train)))
print('Testing Observations: %d weeks' % (len(ts_test)))

fig=display_figure_w_TSs(ts_train, ts_test, 'Training set', 'Test set', 'Training and Test Sets for Modeling')


## General Crime Rate Modeling

<span style="font-size:1.2em; background: lightblue">All modeling of General Crime rate is in the notebook with Part II [part 2 notebook](capstone_prj_scrub_part2-px.ipynb)</span>

## Crime Rate per Offense Category Modeling

### Loading the dictionaries with time-series

In [ ]:
with open('data/pickled_ts/TS_crime_category.pickle', 'rb') as f:
    TS_crime_category=pickle.load(f)
    
with open('data/pickled_ts/TS_crime_against.pickle', 'rb') as f:
    TS_crime_against=pickle.load(f)
        
with open('data/pickled_ts/TS_crime_location.pickle', 'rb') as f:
    TS_crime_location=pickle.load(f)

### Checking for stationarity

#### Checking the stationarity of the time-series in Offense category dictionary

In [ ]:
df_results1, ts_stationary1, ts_non_stationary_diff1=check_stationarity_multiple(TS_crime_category, window=52, plot=True)

In [ ]:
df_results1

<br><br><span style="font-size:1.2em; background:pink">There are **12** time-series that are already stationary and **11** that are not and which require additional processing (differencing).</span><br><br>
    
    

#### Differencing the time series that are not stationary

In [ ]:

df_results2, ts_stationary2, ts_non_stationary_diff2=check_stationarity_multiple(ts_non_stationary_diff1, 
                                                                                  window=52, plot=True)

In [ ]:
df_results2

<br><br><span style="font-size:1.2em; background:lightgreen">All **11** time-series got stationarized by the first dfferencing </span><br><br>
<span style="font-size:1.2em;">Theere are two separate dictionaries for offenses categories: one with 12 original time-series, that were stationary from the get go and another one with 11 time-series that were pre-processed with the first differencing.</span><br><br>
<span style="font-size:1.2em; background:lightblue">The next step is to explore ACFs and PACFs of the timeframe and make a decision on the pdq and PDQs orders.</span><br><br>


    

#### Exploring ACFs and PACFs of the originally stationary time-series

In [ ]:
ACF_PACF_multiple(ts_stationary1);

In [ ]:
ACF_PACF_multiple(ts_stationary2);

### Auto ARIMA for multiple categories of offenses

In [ ]:
# RESULTS = {}
# for crime, ts  in  TS_crime_category.items():
#     crime_categories_results = {}

#     # Splittting it up
#     print('==='*20)
#     print('GRIDSEARCHING FOR {}'.format(crime)) 
#     train_size = round(len(ts) * 0.90)
#     ts_train, ts_test = ts[:train_size], ts[train_size:]


#     predictions_fig=display_figure_w_TSs(ts_train, ts_test, 'Training set', 'Test set', 
#                                              'Training and Test Sets for Modeling {}'.format(crime), limit_=False)


#     #### Gridsearch

#     auto_model_train = pm.auto_arima(ts_train,
#                             start_p=0,start_q=0, d=1,
#                             start_P=1, start_Q=1, D=1,
#                             max_p=2, max_q=2,
#                             max_P=2, max_Q=2,
#                             m=52, maxiter=150,
#                             trace=False,verbose=True)

#     ## Fit SARIMAX with best parmas and compare forecast vs test
#     best_model = tsa.SARIMAX(ts_train,order=auto_model_train.order,
#                                 seasonal_order = auto_model_train.seasonal_order,
#                                 enforce_invertibility=False).fit()

#     ## Use diagnostics
#     diagnostics(best_model)

#     ## Prediction comparison
#     plt.style.use('ggplot')
#     y_hat_train=best_model.predict(typ='levels')
#     y_hat_test=best_model.predict(start=ts_test.index[0], end=ts_test.index[-1], typ='levels')

#     rmse = np.sqrt(mean_squared_error(ts_test, y_hat_test))
#     print('RMSE of the {} model for {}'.format(crime, round(rmse,2)))


#     predictions_fig=display_figure_w_TSs(ts_train, ts_test, 'Train set', 'Test set', 
#                              'Training and Test Sets Raw Values and Predictions, {}'.format(crime),
#                               n=4, ts3=y_hat_test,
#                               ts4=y_hat_train, label3='Prediction for Test set', label4='Prediction for Training set',
#                                          limit_=False)      


#     print('\t-FINAL MODEL:')

#     final_model = tsa.SARIMAX(ts,order=auto_model_train.order,
#                         seasonal_order = auto_model_train.seasonal_order,
#                         enforce_invertibility=False).fit()

#     ## Plot forecast
#     forecast_fig=plot_predictions(ts, final_model, 'Forecast For Two Years Forward, {}'.format(crime),
#                                       steps=104, xmin='2015')

#     ## Fill in results and
#     crime_categories_results['final_model'] = final_model
#     crime_categories_results['predict_fig'] = predictions_fig
#     crime_categories_results['forecast_fig'] = forecast_fig

#     ## Saving results to RESULTS dict
#     RESULTS[crime] = crime_categories_results
        
#     print("\n\n")

In [ ]:
# cropped_RESULTS = {key:val for key, val in RESULTS.items() if ((key != 'Sex Offenses')&(key != 'Weapon Law Violations'))}

In [ ]:
# cropped_RESULTS.keys()

In [ ]:
# with open('data/pickled_models/RESULTS1.pickle', 'wb') as f:
#     pickle.dump(cropped_RESULTS, f)

In [ ]:
# TS_crime_category_to_rerun1={}
# TS_crime_category_to_rerun1['Sex Offenses']=TS_crime_category['Sex Offenses'].copy()

# TS_crime_category_to_rerun1['Weapon Law Violations']=TS_crime_category['Weapon Law Violations'].copy()

In [ ]:
# RESULTS_second_run = {}
# for crime, ts  in  TS_crime_category_to_rerun1.items():
#     crime_categories_results = {}

#     # Splittting it up
#     print('==='*20)
#     print('GRIDSEARCHING FOR {}'.format(crime)) 
#     train_size = round(len(ts) * 0.90)
#     ts_train, ts_test = ts[:train_size], ts[train_size:]


#     predictions_fig=display_figure_w_TSs(ts_train, ts_test, 'Training set', 'Test set', 
#                                              'Training and Test Sets for Modeling {}'.format(crime), limit_=False)


#     #### Gridsearch
#     auto_model_train = pm.auto_arima(ts_train,
#                             start_p=0,start_q=0, d=0,
#                             start_P=0, start_Q=0, D=0,
#                             max_p=2, max_q=2, max_d=1,
#                             max_P=2, max_Q=2, max_D=1,
#                             m=52, maxiter=300,
#                             trace=False,verbose=True)
     

#     ## Fit SARIMAX with best parmas and compare forecast vs test
#     best_model = tsa.SARIMAX(ts_train,order=auto_model_train.order,
#                                 seasonal_order = auto_model_train.seasonal_order,
#                                 enforce_invertibility=False).fit()


#     ## Use diagnostics
#     diagnostics(best_model)

#     ## Prediction comparison
#     plt.style.use('ggplot')
#     y_hat_train=best_model.predict(typ='levels')
#     y_hat_test=best_model.predict(start=ts_test.index[0], end=ts_test.index[-1], typ='levels')

#     rmse = np.sqrt(mean_squared_error(ts_test, y_hat_test))
#     print('RMSE of the {} model for {}'.format(crime, round(rmse,2)))


#     predictions_fig=display_figure_w_TSs(ts_train, ts_test, 'Train set', 'Test set', 
#                              'Training and Test Sets Raw Values and Predictions, {}'.format(crime),
#                               n=4, ts3=y_hat_test,
#                               ts4=y_hat_train, label3='Prediction for Test set', 
#                                          label4='Prediction for Training set', limit_=False)      


#     print('\t-FINAL MODEL:')

#     final_model = tsa.SARIMAX(ts,order=auto_model_train.order,
#                         seasonal_order = auto_model_train.seasonal_order,
#                         enforce_invertibility=False).fit()

#     ## Plot forecast
#     forecast_fig=plot_predictions(ts, final_model, 'Forecast For Two Years Forward, {}'.format(crime),
#                                       steps=104, xmin='2015')

#     ## Fill in results and
#     crime_categories_results['final_model'] = final_model
#     crime_categories_results['predict_fig'] = predictions_fig
#     crime_categories_results['forecast_fig'] = forecast_fig

#     ## Saving results to RESULTS dict
#     RESULTS_second_run[crime] = crime_categories_results
        
#     print("\n\n")

In [ ]:
# with open('data/pickled_models/RESULTS2.pickle', 'wb') as f:
#     pickle.dump(RESULTS_second_run, f)

In [ ]:
# def print_out_models(dictionary):
#     for crime, dict_ in dictionary.items():
#         print('OFFENSE CATEGORY: '+ crime)
#         for key, value in dict_.items():
#             if key=='final_model':
#                 print('\nTHE FINAL MODEL SUMMARY: \n')
#                 display(value.summary());
#                 display(value.plot_diagnostics(figsize=(15,7)));
#             elif key=='predict_fig':
#                 print('\nPREDICTION FOR TRAIN AND TEST sets: \n')
#                 display(value);
#             else:
#                 print('\nFORECAST: \n')
#                 display(value);
#             plt.close() 


In [ ]:
with open('data/pickled_models/RESULTS1.pickle', 'rb') as f:
    results1_back=pickle.load(f)

In [ ]:
with open('data/pickled_models/RESULTS2.pickle', 'rb') as f:
    results2_back=pickle.load(f)

In [ ]:
combined_results = {**results1_back, **results2_back}

In [ ]:
print_out_models(combined_results)

# iNTERPRET

# CONCLUSIONS & RECOMMENDATIONS

> Summarize your conclusions and bullet-point your list of recommendations, which are based on your modeling results.

# TO DO/FUTURE WORK

- 